# Validation of the PyPSA-Africa solar and wind potential

This notebooks investigates the data quality of the African solar and wind potential
data by comparing PyPSA, and other research results.

To reproduce the findings obtained in this notebook,
please run the full snakemake workflow for the Africa.
To do so, please set ``countries = ["Africa"]`` in the ``config.yaml`` file.

Note. An unoptimized prepared network is sufficient for this notebook. 

#### Generate profiles (if not already available)
If you don't have the renewable profiles such as:
- `resources/profile_solar.nc`
- `resources/profile_onwind.nc`
- `resources/profile_offwind-ac.nc`

you could try to generate them from the terminal. In this case set the path so that you are located at ~/pypsa-africa. Once you are there, use the command:

```
snakemake -j 1 resources/profile_{technology}.nc
```

replace {technology} for the timeseries of interest.

## Preparation

### Import packages

In [1]:
import atlite
import cartopy.crs as ccrs
import xarray as xr
import geopandas as gpd
#import holoviews as hv
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
import pypsa
import shutil
from rasterio.plot import show
from atlite.gis import shape_availability, ExclusionContainer


plt.rcParams["figure.figsize"] = [7, 7]
%matplotlib inline

### Set main directory to root folder

In [2]:
# set current folders
import sys

sys.path.append("../../")  # adds path to $ .../pypsa-africa
from scripts._helpers import sets_path_to_root

sets_path_to_root("pypsa-africa")  # moves path to root

## 1. Load and open the PyPSA-Africa generated profiles
The `build_renewable_profiles` for each technology are stored in the `resources` folder. We read first the path and open then with xarray the .nc file. As you can see there is quite a lot data available.  What you should note is that only the `potential` is grided (can be shown in a nice raster on a map). From the `profiles` we can only retrieve timeseries data without mapping it.

In [4]:
solar_path = os.path.realpath("resources") + "/profile_solar.nc"
solar = xr.open_dataset(solar_path)
solar

wind_path = os.path.realpath("resources") + "/profile_onwind.nc"
wind = xr.open_dataset(wind_path)
wind

## 2. Calculate PyPSA-Africa technical potentials
- `profiles` are hourly profiles and given as `per unit` values for each bus (region)
- `per-unit` is scaled according to the `p_nom_max` of each bus (region)
- To calculate the potential across Africa we need to do the following:

In [5]:
technical_solar_potential_twh = sum(solar["p_nom_max"]*solar["profile"]).sum()/10**6  # MWh to TWH conversion
technical_wind_potential_twh = sum(wind["p_nom_max"]*wind["profile"]).sum()/10**6  # MWh to TWH conversion

In [21]:
print(
f"Technical solar potential: {technical_solar_potential_twh.values} \n",
f"Technical wind potential: {technical_wind_potential_twh.values}"
)

Comparing this to other sources.

WIND:
- International Finance Cooperation reports 180.000 TWh technical wind potential for cut-in speed of 6.0 m/s (10.1016/j.renene.2021.07.021)
- International Finance Cooperation reports 118.700 TWh technical wind potential for cut-in speed of 7.5 m/s (10.1016/j.renene.2021.07.021)
- International Finance Cooperation reports 64.700 TWh technical wind potential for cut-in speed of 8.5 m/s (10.1016/j.renene.2021.07.021)
- Mentis et al. (2015) reports 34.000 TWh technical wind potential (10.1016/j.renene.2015.03.072)
- IRENA 2014 reports 66.600 TWh technical potential for turbines with capacity factors above >30%(Title: Estimating the Renewable Energy Potential in Africa. A GIS-based approach) https://www.irena.org/publications/2014/Aug/Estimating-the-Renewable-Energy-Potential-in-Africa-A-GIS-based-approach

SOLAR:
- IRENA 2014 reports 660.000 TWh technical solar PV potential (Title: Estimating the Renewable Energy Potential in Africa. A GIS-based approach) https://www.irena.org/publications/2014/Aug/Estimating-the-Renewable-Energy-Potential-in-Africa-A-GIS-based-approach